In [ ]:
# ---- Global config & paths ----
import pathlib, yaml

ROOT = pathlib.Path().resolve()
CFG = yaml.safe_load(open(parent.ROOT / "configs" / "base.yaml"))

DATA_DIR = ROOT / CFG["paths"]["data_dir"]
WEIGHTS_DIR = ROOT / CFG["paths"]["weights_dir"]
WEIGHTS_DIR.mkdir(parents=True, exist_ok=True)
WEIGHTS_PATH = WEIGHTS_DIR / CFG["paths"]["weights_name"]

RUNS_DIR = ROOT / CFG["paths"]["runs_dir"]
RUNS_DIR.mkdir(parents=True, exist_ok=True)

print("DATA_DIR:", DATA_DIR)
print("WEIGHTS_PATH:", WEIGHTS_PATH)
print("RUNS_DIR:", RUNS_DIR)

IMPORTS, LOAD MODEL, LANDMARKS:
--------------

In [1]:
import mediapipe as mp
import cv2
import numpy as np
import os
from tensorflow.keras.models import load_model

In [2]:
selected_indices = [0, 4, 9, 10, 22, 24, 33, 43, 44, 46, 50, 54, 55, 58, 64, 70, 105, 107, 110, 112, 127, 130, 149, 175, 205, 239, 252, 254, 263, 273, 274, 276, 280, 284, 285, 288, 294, 300, 334, 336, 339, 341, 356, 359, 400, 425, 459, 468, 473]

In [2]:
model = load_model(str(WEIGHTS_PATH))

DEEPFAKE DETECTOR:
-------------

In [6]:
mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, refine_landmarks=True)


def predict_faces(image_path, model, selected_indices):
    image_path = str(DATA_DIR / image_path) if not os.path.isabs(image_path) else image_path
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Image not found.")
        return None

    height, width, _ = image.shape
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    detection_results = face_detection.process(image_rgb)

    predictions = []

    if detection_results.detections:
        for detection in detection_results.detections:
            box = detection.location_data.relative_bounding_box
            x, y, w, h = (int(box.xmin * width), int(box.ymin * height),
                          int(box.width * width), int(box.height * height))

            cropped_face = image[max(y, 0):min(y + h, height),
                                 max(x, 0):min(x + w, width)]
            cropped_face_rgb = cv2.cvtColor(cropped_face, cv2.COLOR_BGR2RGB)
            resized_face = cv2.resize(cropped_face_rgb, (256, 256))

            mesh_results = face_mesh.process(resized_face)
            if mesh_results.multi_face_landmarks:
                for landmarks in mesh_results.multi_face_landmarks:
                    landmark_array = np.array([(lm.x, lm.y, lm.z) for lm in landmarks.landmark])
                    selected_landmarks = landmark_array[selected_indices]

                    if selected_landmarks.shape[0] != 49:
                        print("Error: Could not extract the required number of landmarks.")
                        continue

                    # Add batch dimension
                    resized_face_batch = np.expand_dims(resized_face, axis=0)
                    selected_landmarks_batch = np.expand_dims(selected_landmarks, axis=0)

                    # Prepare input for the model and predict
                    prediction = model.predict([resized_face_batch, selected_landmarks_batch])
                    label = "Real" if prediction < 0.5 else "Fake"
                    predictions.append((x, y, w, h, label))
            else:
                print("Error: Could not extract any landmarks.")

    return predictions


I0000 00:00:1717606065.610630 2422624 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
I0000 00:00:1717606065.623817 2422624 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1


In [ ]:
predictions = predict_faces("two_faces_test.jpg", model, selected_indices)